# Implementando um classificador baseado em distância com um circuito de interferência quântica
##### [Maria Schuld, Mark Fingerhuth, Francesco Petruccione](https://arxiv.org/abs/1703.10793)

O classificador quântico aqui disposto utiliza a interferência quântica para medir a distância de um vetor de entrada para cada
um dos vetores de treinamento e assim classificá-lo. Como prova de conceito, é montado um circuito quântico para classificar 
instâncias do conjuto de dados [Iris](http://archive.ics.uci.edu/ml/datasets/Iris).

A ideia principal é utilizar a interferência quântica para medir a distância de um novo vetor de entrada para cada um dos 
vetores de treinamento disponíveis e os classificar, como prova de conceito, é montado um circuito quântico para classificar 
instâncias do dataset Iris. São expostos resultados preditos, simulados e a partir de execuções reais em computadores quânticos 
da IBM, que só permitem a implementação de 80 portas quânticas, mas simulações numéricas mostram que o classificador funciona 
bem o suficiente com as tarefas de referência.

## Circuito

![title](circuit.jpg)

$\newcommand{\ket}[1]{\left\vert #1 \right\rangle}$
$\psi_A = \dfrac{1}{2} \left( \ket{0000} + \ket{0100} + \ket{1000} + \ket{1100} \right)$

In [ ]:
$\psi_B = \dfrac{1}{2} \left( \ket{1000} + \ket{1100} + \cos{\dfrac{\theta}{2}}\ket{0000} + \sin{\dfrac{\theta}{2}}\ket{0010} + 
\cos{\dfrac{\theta}{2}}\ket{0100} + \sin{\dfrac{\theta}{2}}\ket{0110} \right)$

$\psi_C = \dfrac{1}{2} \left( \ket{1100} + \cos{\dfrac{\phi_0}{2}}\ket{1000} + \sin{\dfrac{\phi_0}{2}}\ket{1010} + \cos{\dfrac{\theta}{2}}\ket{0100} + \sin{\dfrac{\theta}{2}}\ket{0110} + \cos{\dfrac{\theta}{2}}\ket{0000} + \sin{\dfrac{\theta}{2}}\ket{0010} \right)$

$\psi_D = \dfrac{1}{2} \left( \cos{\dfrac{\phi_1}{2}}\ket{1100} + \sin{\dfrac{\phi_1}{2}}\ket{1110} + \cos{\dfrac{\phi_0}{2}}\ket{1000} + \sin{\dfrac{\phi_0}{2}}\ket{1010} + \cos{\dfrac{\theta}{2}}\ket{0100} + \sin{\dfrac{\theta}{2}}\ket{0110} + \cos{\dfrac{\theta}{2}}\ket{0000} + \sin{\dfrac{\theta}{2}}\ket{0010} \right)$

$\psi_E = \dfrac{1}{2} \left( \cos{\dfrac{\phi_1}{2}}\ket{1101} + \sin{\dfrac{\phi_1}{2}}\ket{1111} + 
\cos{\dfrac{\phi_0}{2}}\ket{1000} + \sin{\dfrac{\phi_0}{2}}\ket{1010} + \cos{\dfrac{\theta}{2}}\ket{0101} + 
\sin{\dfrac{\theta}{2}}\ket{0111} + \cos{\dfrac{\theta}{2}}\ket{0000} + \sin{\dfrac{\theta}{2}}\ket{0010} \right)$

$\psi_F = \dfrac{1}{2} \left( \left( \dfrac{\ket{0} - \ket{1}}{\sqrt{2}} \right) \left( \cos{\dfrac{\phi_1}{2}}\ket{101} + \sin{\dfrac{\phi_1}{2}}\ket{111} + \cos{\dfrac{\phi_0}{2}}\ket{000} + \sin{\dfrac{\phi_0}{2}}\ket{010} \right) + \left( \dfrac{\ket{0} + \ket{1}}{\sqrt{2}} \right) \left( \cos{\dfrac{\theta}{2}}\ket{101} + \sin{\dfrac{\theta}{2}}\ket{111} + \cos{\dfrac{\theta}{2}}\ket{000} + \sin{\dfrac{\theta}{2}}\ket{010} \right) \right)$

## Implementação

In [ ]:
# Install Libs
import sys
!{sys.executable} -m pip install --user numpy
!{sys.executable} -m pip install --user matplotlib
!{sys.executable} -m pip install --user qiskit
!{sys.executable} -m pip install --user scikit-learn

In [ ]:
# Import Libs
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from qiskit import *
import qutils
import iris
import math

In [ ]:
# Auxiliary function definitions
def calculate_theta(sample):        
    if sample[0] < 0 and sample[1] < 0:
        value = sample[1]
        theta = math.acos(value)*2+math.pi
    elif sample[1] < 0:
        value = sample[1]
        theta = math.asin(value)*2
    else:
        value = sample[0]
        theta = math.acos(value)*2
    
    return theta

Carregar os dados do dataset Iris e indexar o vetor da  entrada


In [ ]:
# Preprocessing
classes = [0,1]
features = [0,1]
input_vector_index = 28
training_vector_indexes = [33, 85]

X, y = iris.load_dataset(classes=classes, features=features)
X = iris.preprocess(X)

input_vector_theta = calculate_theta(X[input_vector_index])
training_vector_thetas = [
    calculate_theta(X[training_vector_indexes[0]]),
    calculate_theta(X[training_vector_indexes[1]])
]

Inicializando os 4 qubits necessários no circuito, que armazenam, a ancilla, o index, o vetor do dataset e a classe

In [ ]:
# Create Circuit
q = QuantumRegister(4, 'q')
c = ClassicalRegister(2, 'c')
qc = QuantumCircuit(q, c)

A qubit da ancilla e o index são colocados em superposição com a aplicação do hadamard, onde o qbit da ancilla interfere 
nas cópias das novas entradas e das novas entradas de treinamento

In [ ]:
# Subcircuit A
qc.h(q[[0,1]])

qc.barrier()
qutils.wavefunction(qc)

É feita uma aplicação condicional do Ry (cnot) e em seguida é aplicado X no qbit da ancilla onde ele é emaranhado com o vetor
dos dados de entrada e assim o conteúdo desse vetor é codificado no qbit da ancilla 

In [ ]:
# Subcircuit B
ry_input = input_vector_theta / 2.0
qc.cx(q[0], q[2])
qc.ry((-1 * ry_input), q[2])
qc.cx(q[0], q[2])
qc.x(q[0])
qc.ry(ry_input, q[2])

qc.barrier()
qutils.wavefunction(qc)

Uma porta Toffoli e em seguida um X é aplicada e o vetor de treinamendo $ x^0 $ é emaranhado com o estado inicial do qbit do 
index e o estado excitado do qbit da ancilla e então o conteúdo do vetor $ x^0 $ é codificado 


In [ ]:
# Subcircuit C
ry_input = training_vector_thetas[0] / 4.0
qc.ccx(q[0], q[1], q[2])
qc.cx(q[1], q[2])
qc.ry(ry_input, q[2])
qc.cx(q[1], q[2])
qc.ry((-1 * ry_input), q[2])
qc.ccx(q[0], q[1], q[2])
qc.cx(q[1], q[2])
qc.ry((-1 * ry_input), q[2])
qc.cx(q[1], q[2])
qc.ry(ry_input, q[2])

qc.x(q[1])

qc.barrier()
qutils.wavefunction(qc)

O vetor de treinamento $ x^1 $ é emaranhado como estado excitado da ancilla e o qbit do index aplicando o Ry condicionalmente
no index qbit usando como parâmetro a entrada de treinamento $ x^1 $

In [ ]:
# Subcircuit D
ry_input = training_vector_thetas[1] / 4.0
qc.ccx(q[0], q[1], q[2])
qc.cx(q[1], q[2])
qc.ry(ry_input, q[2])
qc.cx(q[1], q[2])
qc.ry((-1 * ry_input), q[2])
qc.ccx(q[0], q[1], q[2])
qc.cx(q[1], q[2])
qc.ry((-1 * ry_input), q[2])
qc.cx(q[1], q[2])
qc.ry(ry_input, q[2])

qc.barrier()
qutils.wavefunction(qc)

In [ ]:
# Subcircuit E
qc.cx(q[1], q[3])

qc.barrier()
qutils.wavefunction(qc)

In [ ]:
Um hadamard é aplicado para que a possibilidade da medição (que será feita no próximo passo) seja 0 aumente 

In [ ]:
# Subcircuit F
qc.h(q[0])

qutils.wavefunction(qc)

São feitas duas medições, uma no qbit da ancilla e outra que é condicional, caso o resultado da medição do qbit da ancilla seja
0 o qbit do index é medido e então a classe será obtida

In [ ]:
# Measure
qc.measure(q[0],c[0])
qc.barrier()
qc.measure(q[3],c[1])

qutils.wavefunction(qc)

In [ ]:
# Print circuit
qc.draw(output = 'mpl')

In [ ]:
qutils.measurement(qc, shots = 1024)

In [ ]:
# Simulate
backend = BasicAer.get_backend('qasm_simulator')
counts = execute(qc, backend, shots=1024).result().get_counts(qc)

In [ ]:
# Postselection probability
pacc = (counts['00'] + counts['10']) / sum(counts.values())
visualization.plot_histogram({ 'P': pacc, '1 - P': (1 - pacc) })